In [3]:
import numpy as np

# Data frå all_pred
parties = {
    "Ap": (19.232083, 18.140119, 20.285584),
    "SV": (9.289484, 8.603512, 9.989698),
    "Sp": (5.593741, 4.456879, 6.828874),
    "Rodt": (5.132772, 4.371419, 5.918580),
    "MDG": (3.416563, 3.022827, 3.769606),
    "Hoyre": (18.504629, 16.695958, 20.438473),
    "Frp": (25.443152, 24.167373, 26.605551),
    "KrF": (3.486558, 3.245975, 3.772522),
    "Venstre": (5.996908, 5.502743, 6.529766)
}

# Antall simuleringar
n_sim = 100000

# Trekke stemmer frå normalfordeling
red_green_votes = np.zeros(n_sim)
blue_votes = np.zeros(n_sim)

for party, (yhat, lower, upper) in parties.items():
    std_dev = (upper - lower) / 3.92  # Omformar intervall til standardavvik (~95% dekking)
    samples = np.random.normal(yhat, std_dev, n_sim)
    
    if party in ["Ap", "SV", "Sp", "Rodt", "MDG"]:
        red_green_votes += samples
    else:
        blue_votes += samples

# Sannsyn for at den raudgrøne blokka er større
prob_red_green_wins = np.mean(red_green_votes > blue_votes)

print(f"Sannsyn for at Ap+SV+Sp+R+MDG > H+FrP+KrF+V: {prob_red_green_wins:.4f}")


Sannsyn for at Ap+SV+Sp+R+MDG > H+FrP+KrF+V: 0.0000


In [4]:
import numpy as np
import pandas as pd

# Input Data (party, estimate, lower, high)
data = {
    'party': ['Ap', 'Hoyre', 'Frp', 'SV', 'Sp', 'KrF', 'Venstre', 'MDG', 'Rodt', 'Andre'],
    'estimate': [19.232083, 18.504629, 25.443152, 9.289484, 5.593741, 3.486558, 5.996908, 3.416563, 5.132772, 3.904109],
    'lower': [17.403175, 15.660473, 23.630302, 8.087047, 3.795392, 3.095527, 5.218467, 2.852166, 4.021895, 2.846611],
    'high': [20.973558, 21.245416, 27.368616, 10.310349, 7.308616, 3.928326, 6.835968, 3.949766, 6.305181, 4.983875]
}

df = pd.DataFrame(data)

# Create new columns to estimate standard deviation using the formula (upper - lower) / 3.92
df['std_dev'] = (df['high'] - df['lower']) / 3.92

# Define groups:
group_1_parties = ['Ap', 'SV', 'Sp', 'Rodt', 'MDG']  # Group 1: Ap, SV, Sp, Rodt, MDG
group_2_parties = ['Hoyre', 'Frp', 'KrF', 'Venstre']  # Group 2: Hoyre, Frp, KrF, Venstre

# Number of simulations
num_simulations = 10000

# Simulation function
def monte_carlo_simulation(df, group_1_parties, group_2_parties, num_simulations):
    group_1_sums = []
    group_2_sums = []

    # Run the Monte Carlo simulations
    for _ in range(num_simulations):
        group_1_sum = 0
        group_2_sum = 0

        # Generate random outcomes for each party in Group 1 and Group 2
        for index, row in df.iterrows():
            party = row['party']
            random_value = np.random.normal(loc=row['estimate'], scale=row['std_dev'])

            if party in group_1_parties:
                group_1_sum += random_value
            elif party in group_2_parties:
                group_2_sum += random_value

        group_1_sums.append(group_1_sum)
        group_2_sums.append(group_2_sum)

    return group_1_sums, group_2_sums

# Run the simulation
group_1_sums, group_2_sums = monte_carlo_simulation(df, group_1_parties, group_2_parties, num_simulations)

# Calculate the probability that Group 1 is larger than Group 2
group_1_larger = np.sum(np.array(group_1_sums) > np.array(group_2_sums)) / num_simulations

# Print results
print(f'Probability that Group 1 (Ap + SV + Sp + Rodt + MDG) is larger than Group 2 (Hoyre + Frp + KrF + Venstre): {group_1_larger:.4f}')


Probability that Group 1 (Ap + SV + Sp + Rodt + MDG) is larger than Group 2 (Hoyre + Frp + KrF + Venstre): 0.0000
